# Lecture 6-1

Multinomial classification  <br>
보기를 2개이상으로 예측을 해보자

예를 들어 a , b , c 라고 하면<br>
a or not <br>
b or not <br>
c or not <br>  이런식으로 배운 것을 다 합친다.

[wA1 wA2 wA3] [x1]=[wA1x1+wA2x2+wA3x3]=[yA] <br>
[wB1 wB2 wB3] [x2]=[wB1x1+wB2x2+wB3x3]=[yB] <br>
[wC1 wC2 wC3] [x3]=[wC1x1+wC2x2+wC3x3]=[yC] <br>

긴 매트릭스를 만들어서 곱셈공식으로 하여 구현

wx = y 는 2.0/1.0/0.1 세개중에서 하나로 나온다. <br>
하지만 sigmoid 는 0~1 사이의 값으로 나와야한다. <br>
<변경><br>
a = 0.7 / b = 0.2 / c = 0.1  총 합은 1<br>
이것이 softmax 라는 함수

'one-hot encoding' 사용하여 확률이 높은 것을 1 로 하고 낮은 것을 0으로 변경

cross-entropy 라는 함수로 실제값과 softmax 로 구한 값의 차이를 구해줌  즉  cost 의 값

실제 값이랑 예측 값이랑 맞으면 cost값이 0이고<br>
틀리면 무한히 값이 커진다.<br>

---

In [1]:
import tensorflow as tf

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x_data = [[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]
# ont_hot_encoding 이고 하나만 1로 하여 구분을 한다
X= tf.placeholder("float",[None,4])
Y= tf.placeholder("float",[None,3])
nb_classes =3

W = tf.Variable(tf.random_normal([4,nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
#shape 주의
hypothesis = tf.nn.softmax(tf.matmul(X, W)+b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis),axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y:y_data})
        if step % 200 ==0:
            print(step,sess.run(cost,feed_dict={X:x_data, Y:y_data}))

0 2.095614
200 0.61765397
400 0.5242207
600 0.43618065
800 0.34824982
1000 0.26159832
1200 0.22220655
1400 0.20271719
1600 0.18623245
1800 0.1721226
2000 0.15992069


arg_max = 몇번째 있는것이 높은 건지 물어봄

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    all = sess.run(hypothesis, feed_dict={X : [[1,11,7,9],[1,3,4,3],[1,1,0,1]]})
    print(all, sess.run(tf.arg_max(all,1)))

[[6.4962758e-03 9.9350375e-01 3.5924734e-15]
 [9.4727063e-01 5.2466612e-02 2.6277208e-04]
 [6.6132313e-01 2.9340395e-01 4.5272917e-02]] [1 0 0]


In [1]:
# 1번째 인덱스 값이 제일 높고 
# 0번째 인덱스 값이 제일 높다.
# 0번쨰 인덱스 값이 제일 높다

---

cross_ebtropy , one_hot , reshape

logits=tf.matmul(X, W)+b  #logits 을 scores 라고 부른다 <br>
#matmul 은 scores를 넣으면 확률로 나오게 되는 함수<br>
hypothesis = tf.nn.softmax(logits)


cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis),axis=1))<Br>
이렇게 하는 방법보다는
    cost_i = tf.nn.softmax_cross_entropy_<b>with_logits(logits=logits</b> , labels=Y_one_hot)<Br>
cost = tf.reduce_mean(cost_i)

In [16]:
import tensorflow as tf
import numpy as np

In [1]:
# 이 동물이 어떠한 종류인지 분류 (0~6 7가지의 종류로 예측)
xy = np.loadtxt('zoo.csv',delimiter=',',dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:,[-1]]

NameError: name 'np' is not defined

In [18]:
nb_classes =7 #7가지의 종류

X = tf.placeholder(tf.float32, [None,16])
Y = tf.placeholder(tf.int32, [None,1]) #shape(? , 1)  [[0],[3]]
Y_one_hot = tf.one_hot(Y, nb_classes)  #shape(? , 1, 7) [[[1000000]],[[0001000]]]
#지금 데이터에는 0~6으로 되어있음 
# one_hot 함수로 하나만 1로 되도록 변경해야함
Y_one_hot = tf.reshape(Y_one_hot,[-1, nb_classes]) # shape(?,7) [[10000000],[0001000]]
 # reshape 차원을 바꿔줌

In [20]:
W = tf.Variable(tf.random_normal([16, nb_classes]),name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X,W)+b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [32]:
#예측한 값이 맞은건지 틀린건지 알아보기 위해
prediction = tf.argmax(hypothesis,1) #hypothesis를 레벨로 변경
correct_prediction = tf.equal(prediction,tf.argmax(Y_one_hot,1)) #one_hot한 거랑 비교
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))# 맞은것을 모아서 평균을 구함 = 정확도

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y:y_data})
        if step % 100 ==0:
            loss, acc = sess.run([cost,accuracy],feed_dict={X: x_data, Y:y_data})
            
            print("Step: {:5}\tLoss: {:.3f}\tACC:{:.2%}".format(
            step, loss, acc))
            
    pred = sess.run(prediction, feed_dict={X: x_data})
    #flatten = [[1],[0]] -> [1,0]
    #zip 각각의 리스트를 zip으로 묶어서 넘김
    for p , y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p==int(y),p,int(y)))

Step:     0	Loss: 5.771	ACC:8.91%
Step:   100	Loss: 0.462	ACC:80.20%
Step:   200	Loss: 0.248	ACC:85.15%
Step:   300	Loss: 0.180	ACC:88.12%
Step:   400	Loss: 0.145	ACC:87.13%
Step:   500	Loss: 0.123	ACC:87.13%
Step:   600	Loss: 0.107	ACC:88.12%
Step:   700	Loss: 0.096	ACC:89.11%
Step:   800	Loss: 0.087	ACC:89.11%
Step:   900	Loss: 0.080	ACC:90.10%
Step:  1000	Loss: 0.074	ACC:91.09%
Step:  1100	Loss: 0.070	ACC:91.09%
Step:  1200	Loss: 0.065	ACC:91.09%
Step:  1300	Loss: 0.062	ACC:91.09%
Step:  1400	Loss: 0.059	ACC:91.09%
Step:  1500	Loss: 0.056	ACC:91.09%
Step:  1600	Loss: 0.054	ACC:91.09%
Step:  1700	Loss: 0.052	ACC:91.09%
Step:  1800	Loss: 0.050	ACC:91.09%
Step:  1900	Loss: 0.048	ACC:91.09%
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 4 True Y: 4
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 4 True Y: 4
[True] Prediction: 4 True Y: 4
[True] Prediction: 1 T